# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [1]:

from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-138712
Azure region: southcentralus
Subscription id: 61c5c3f0-6dc7-4ed9-a7f3-c704b20e3b30
Resource group: aml-quickstarts-138712


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
compute_cluster_name= "cluster-one"
vm_size="Standard_D2_V2"
max_nodes=4
#Verify that cluster does not already exist
try:
    compute_target=ComputeTarget(workspace=ws, name=compute_cluster_name)
    print('Cluster found with the name {0}.. Proceed'.format(compute_cluster_name))
except ComputeTargetException:
    print('Creating a new cluster with the name {0}'.format(compute_cluster_name))
    compute_config=AmlCompute.provisioning_configuration(vm_size=vm_size,max_nodes=max_nodes)
    compute_target=ComputeTarget.create(ws,compute_cluster_name,compute_config)

compute_target.wait_for_completion(show_output=True)

Creating a new cluster with the name cluster-one
Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [4]:

#Define RunConfig for the compute
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import BayesianParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform,choice
import os

# Specify parameter sampler (continuous)
ps = BayesianParameterSampling ({
    "--C":choice(0.00001,0.0001,0.001,0.01,0.1,1,10,100,200,500,1000),
    "--max_iter":choice(50,100,200,300,500,1000)
})

if "training" not in os.listdir():
    os.mkdir("./training")

#Define primary metric goal
#goal=primary_metric_goal("MAXIMIZE")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory='.',
compute_target=compute_target,vm_size=vm_size,vm_priority="lowpriority",entry_script="train.py")

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.

hyperdrive_config = HyperDriveConfig(
    estimator=est,
    hyperparameter_sampling=ps,
    primary_metric_name='Accuracy',
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=100,
    max_concurrent_runs=4
)

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.


In [5]:
#TODO: Submit your experiment

from azureml.widgets import RunDetails
hyperdrive_run=exp.submit(hyperdrive_config)
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_8aaf9486-dffa-46ba-80b9-d4332eae95bb
Web View: https://ml.azure.com/experiments/udacity-project/runs/HD_8aaf9486-dffa-46ba-80b9-d4332eae95bb?wsid=/subscriptions/61c5c3f0-6dc7-4ed9-a7f3-c704b20e3b30/resourcegroups/aml-quickstarts-138712/workspaces/quick-starts-ws-138712

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-02-12T17:14:39.613720][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2021-02-12T17:14:39.761298][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n""<START>[2021-02-12T17:14:39.106558][API][INFO]Experiment created<END>\n"<START>[2021-02-12T17:14:40.1849435Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>

Execution Summary
RunId: HD_8aaf9486-dffa-46ba-80b9-d4332eae95bb
Web View: https://ml.azure.com/experiments/udacity-project/runs/HD_8aaf9486-dffa-46ba-80b9-d4332eae95bb?wsid=/subscriptions/61c5c

{'runId': 'HD_8aaf9486-dffa-46ba-80b9-d4332eae95bb',
 'target': 'cluster-one',
 'status': 'Canceled',
 'startTimeUtc': '2021-02-12T17:14:38.900436Z',
 'endTimeUtc': '2021-02-12T17:24:54.65829Z',
 'error': {'error': {'code': 'UserError',
   'message': 'User errors were found in at least one of the child runs.',
   'messageParameters': {},
   'details': []},
  'time': '0001-01-01T00:00:00.000Z'},
 'warnings': [{'source': 'SecondaryError',
   'message': '{\n  "error": {\n    "code": "UserError",\n    "severity": null,\n    "message": "User errors were found in at least one of the child runs.",\n    "messageFormat": null,\n    "messageParameters": {},\n    "referenceCode": null,\n    "detailsUri": null,\n    "target": null,\n    "details": [],\n    "innerError": null,\n    "debugInfo": null\n  },\n  "correlation": null,\n  "environment": null,\n  "location": null,\n  "time": "0001-01-01T00:00:00+00:00",\n  "componentName": null\n}'}],
 'properties': {'primary_metric_config': '{"name": "Acc

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
import joblib
# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])
print('\n Best Run Metrics:',best_run_metrics)
print('\n Best Run Details:',best_run.get_details())
print('\n Best Run File names:',best_run.get_file_names())
### YOUR CODE HERE ###
model=best_run.register_model(model_name='hyperdrive_model',model_path='./outputs/model.joblib')
#joblib.dump(value=model,filename='./outputs/model.joblib')

In [ ]:
#TODO: Save the best model

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service